In [ ]:
!pip install llama-index pypdf sentence_transformers -q docx2txt

Using OpenAI GPT-3 model

In [ ]:
import os
import openai

openai.api_key = ""
os.environ["OPENAI_API_KEY"] = ""


Indexing

In [ ]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader
# load document from a directory
documents = SimpleDirectoryReader('test').load_data()

# create an index from the documents
index = VectorStoreIndex.from_documents(documents)

# create a query engine from the index
query_engine = index.as_query_engine()

# Query the engine
response = query_engine.query("What is the letter about?")
print(response)

Saving and loading index

In [ ]:
index.storage_context.persist("naval_index")

In [ ]:
from llama_index import StorageContext, load_index_from_storage

# rebuild storage context
storage_context = StorageContext.from_defaults(persist_dir="naval_index")
# load index
new_index = load_index_from_storage(storage_context)

new_query_engine = new_index.as_query_engine()
response = new_query_engine.query("who is this text about?")
print(response)


Customizing LLMs using LangChain

In [ ]:
from llama_index import LLMPredictor, ServiceContext
from langchain.chat_models import ChatOpenAI

# Create a predictor using a custom model
llm_predictor = LLMPredictor(llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo"))

# Create a service context with the custom predictor
service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor)

# Create an index using the service context
custom_llm_index = VectorStoreIndex.from_documents(
    documents, service_context=service_context
)

custom_llm_query_engine = custom_llm_index.as_query_engine()
response = custom_llm_query_engine.query("who is this text about?")
print(response)

Custom Prompt
for more structured responses

In [ ]:
from llama_index import Prompt

# Define a custom prompt
template = (
    "We have provided context information below. \n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "Given this information, please answer the question and each answer should start with code word AI Demos: {query_str}\n"
)
qa_template = Prompt(template)

# Use the custom prompt when querying
query_engine = custom_llm_index.as_query_engine(text_qa_template=qa_template)
response = query_engine.query("who is this text about?")
print(response)


Custom embedding

In [ ]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import LangchainEmbedding, ServiceContext

# Load in a specific embedding model
embed_model = LangchainEmbedding(HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2'))

# Create a service context with the custom embedding model
service_context = ServiceContext.from_defaults(embed_model=embed_model)

# Create an index using the service context
new_index = VectorStoreIndex.from_documents(
    documents,
    service_context=service_context,
)

query_engine = new_index.as_query_engine()
response = query_engine.query("list 5 important points from this letter")
print(response)